# Eenvoudige Beeldclassifier

Deze notebook laat zien hoe je afbeeldingen kunt classificeren met behulp van een vooraf getraind neuraal netwerk.

**Wat je zult leren:**
- Hoe je een vooraf getraind model laadt en gebruikt
- Beeldvoorverwerking
- Voorspellingen maken op afbeeldingen
- Begrijpen van vertrouwensscores

**Toepassing:** Objecten in afbeeldingen identificeren (zoals "kat", "hond", "auto", enz.) 

---


## Stap 1: Vereiste bibliotheken importeren

Laten we de benodigde tools importeren. Maak je geen zorgen als je ze nog niet allemaal begrijpt!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Stap 2: Vooraf getraind model laden

We gebruiken **MobileNetV2**, een neuraal netwerk dat al is getraind op miljoenen afbeeldingen.

Dit wordt **Transfer Learning** genoemd - het gebruiken van een model dat door iemand anders is getraind!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Stap 3: Hulpfuncties

Laten we functies maken om afbeeldingen te laden en voor te bereiden voor ons model.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Stap 4: Testen op voorbeeldafbeeldingen

Laten we enkele afbeeldingen van het internet classificeren!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Classificeer Elke Afbeelding


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Stap 5: Probeer je eigen afbeeldingen!

Vervang de onderstaande URL door een willekeurige afbeeldings-URL die je wilt classificeren.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Wat is er net gebeurd?

1. **We hebben een voorgetraind model geladen** - MobileNetV2 is getraind op miljoenen afbeeldingen
2. **We hebben afbeeldingen vooraf verwerkt** - Ze zijn aangepast en geformatteerd voor het model
3. **Het model heeft voorspellingen gedaan** - Het gaf waarschijnlijkheden voor 1000 objectklassen
4. **We hebben de resultaten gedecodeerd** - Nummers zijn omgezet naar menselijk leesbare labels

### Begrijpen van vertrouwensscores

- **90-100%**: Zeer zeker (bijna zeker correct)
- **70-90%**: Zeker (waarschijnlijk correct)
- **50-70%**: Enigszins zeker (misschien correct)
- **Onder 50%**: Niet erg zeker (onzeker)

### Waarom kunnen voorspellingen fout zijn?

- **Ongewone hoek of belichting** - Het model is getraind op typische foto's
- **Meerdere objecten** - Het model verwacht één hoofdobject
- **Zeldzame objecten** - Het model kent slechts 1000 categorieën
- **Lage kwaliteit afbeelding** - Vage of gepixelde afbeeldingen zijn moeilijker


## 🚀 Volgende stappen

1. **Probeer verschillende afbeeldingen:**
   - Zoek afbeeldingen op [Unsplash](https://unsplash.com)
   - Rechtsklik → "Afbeeldingsadres kopiëren" om de URL te verkrijgen

2. **Experimenteer:**
   - Wat gebeurt er met abstracte kunst?
   - Kan het objecten vanuit verschillende hoeken herkennen?
   - Hoe gaat het om met meerdere objecten?

3. **Meer leren:**
   - Verken [lessen over Computer Vision](../lessons/4-ComputerVision/README.md)
   - Leer hoe je je eigen beeldclassifier kunt trainen
   - Begrijp hoe CNNs (Convolutionele Neurale Netwerken) werken

---

## 🎉 Gefeliciteerd!

Je hebt zojuist een beeldclassifier gebouwd met behulp van een geavanceerd neuraal netwerk!

Deze techniek wordt ook gebruikt voor:
- Google Foto's (het organiseren van je foto's)
- Zelfrijdende auto's (het herkennen van objecten)
- Medische diagnoses (het analyseren van röntgenfoto's)
- Kwaliteitscontrole (het opsporen van defecten)

Blijf ontdekken en leren! 🚀



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
